# Base de dados MNIST

- Imagens: https://github.com/myleott/mnist_png

In [2]:
import numpy as np
# para descompartar o arquivo tar.gz
import tarfile 
import urllib # para download de arquivos
import mxnet as mx # framework para aprendizado de máquina
import json

In [3]:
#descompactar 
base = tarfile.open('mnist_png.tar.gz') 
# extrair para o direitório padrão 
base.extractall('./')
# para liberar memoria
base.close()

In [4]:
# download de script para rec
# https://awstip.com/make-recordio-files-for-training-on-amazon-sagemaker-a5fe2e6c22eb
script_url = "https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py"
urllib.request.urlretrieve(script_url, 'im2rec.py')

('im2rec.py', <http.client.HTTPMessage at 0x7f4c3c644d30>)

In [5]:
# será criado um arquivo extensão lst depois criamos para rec
!python im2rec.py --list --recursive train_data_mnist mnist_png/training

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9


In [3]:
!python im2rec.py --list --recursive test_data_mnist mnist_png/testing

0 0
0/.ipynb_checkpoints 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10


In [4]:
!python im2rec.py --num-thread 2 train_data_mnist mnist_png/training

Creating .rec file from /root/rede-neural/train_data_mnist.lst in /root/rede-neural
time: 0.0346677303314209  count: 0
time: 4.6495583057403564  count: 1000
time: 4.347126245498657  count: 2000
time: 4.272626638412476  count: 3000
time: 4.144980430603027  count: 4000
time: 4.1235620975494385  count: 5000
time: 4.228705406188965  count: 6000
time: 4.308286905288696  count: 7000
time: 4.226806402206421  count: 8000
time: 4.246553421020508  count: 9000
time: 4.226423263549805  count: 10000
time: 4.88622784614563  count: 11000
time: 4.355528354644775  count: 12000
time: 4.306909084320068  count: 13000
time: 4.211307525634766  count: 14000
time: 4.283477306365967  count: 15000
time: 4.270052194595337  count: 16000
time: 5.528489828109741  count: 17000
time: 4.090841293334961  count: 18000
time: 4.273306608200073  count: 19000
time: 4.1561267375946045  count: 20000
time: 4.203909397125244  count: 21000
time: 4.123006343841553  count: 22000
time: 4.2314772605896  count: 23000
time: 4.16663026

In [5]:
!python im2rec.py --num-thread 2 test_data_mnist mnist_png/testing

Creating .rec file from /root/rede-neural/test_data_mnist.lst in /root/rede-neural
time: 0.024582624435424805  count: 0
time: 2.939357280731201  count: 1000
time: 3.870227575302124  count: 2000
time: 3.8115968704223633  count: 3000
time: 3.9712321758270264  count: 4000
time: 4.171115875244141  count: 5000
time: 3.9943861961364746  count: 6000
time: 4.227914094924927  count: 7000
time: 3.837383985519409  count: 8000
time: 3.8401660919189453  count: 9000
time: 3.909902334213257  count: 10000


# Configurações do sagemaker

In [6]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [15]:
bucket = 'redeneural'
subpasta_dataset = 'mnist'
role = get_execution_role()
s3_treinamento = 's3://{}/{}/train'.format(bucket, subpasta_dataset)
s3_teste = 's3://{}/{}/test'.format(bucket, subpasta_dataset)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_dataset)
print('Localização da base de treinamento: ', s3_treinamento)
print('Localização da base de teste: ', s3_teste)
print('Localização do modelo: ', output_location)

Localização da base de treinamento:  s3://redeneural/mnist/train
Localização da base de teste:  s3://redeneural/mnist/test
Localização do modelo:  s3://redeneural/mnist/output


# Modelo de rede neural

In [16]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
container = sagemaker.image_uris.retrieve(framework = 'image-classification',
                                          region = boto3.Session().region_name)

In [17]:
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
rede_neural = sagemaker.estimator.Estimator(image_uri = container,
                                            role = role,
                                            instance_count = 1,
                                            instance_type = 'ml.p3.2xlarge',
                                            input_mode = 'File', # para extensão arquivo rec
                                            output_path = output_location)

In [18]:
# image_shape esse treinamento tem somente um canal de cor 
# https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html
rede_neural.set_hyperparameters(num_layers = 110,
                                image_shape = '1,28,28',
                                num_classes = 10,
                                num_training_samples = 50000,
                                mini_batch_size = 128,
                                epochs = 10,
                                learning_rate = 0.001)       

In [19]:
train_data = sagemaker.inputs.TrainingInput(s3_treinamento,
                                            content_type = 'application/x-recordio',
                                            s3_data_type = 'S3Prefix')
test_data = sagemaker.inputs.TrainingInput(s3_teste,
                                            content_type = 'application/x-recordio',
                                            s3_data_type = 'S3Prefix')

In [20]:
rede_neural.fit({'train': train_data, 'validation': test_data})

2023-04-07 18:07:11 Starting - Starting the training job...
2023-04-07 18:07:35 Starting - Preparing the instances for trainingProfilerReport-1680890831: InProgress
......
2023-04-07 18:08:40 Downloading - Downloading input data...
2023-04-07 18:09:00 Training - Downloading the training image..................
2023-04-07 18:12:06 Training - Training image download completed. Training in progress....Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Fri Apr  7 18:12:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.07    Driver Version: 515.65.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | G

# Deploy

In [21]:
rede_neural_classificador = rede_neural.deploy(initial_instance_count=1, instance_type = 'ml.m4.xlarge')

------------!

In [ ]:
imagem = open('digit.png', 'rb').read()

In [ ]:
runtime = boto3.Session().client(service_name = 'sagemaker-runtime')
resposta = runtime.invoke_endpoint(EndpointName = 'image-classification-2023-04-07-18-20-01-871', 
                                   ContentType = 'application/x-image', 
                                   Body=bytearray(imagem))

In [ ]:
resposta

In [ ]:
resultado = resposta['Body'].read()
resultado = json.loads(resultado)
print(resultado)

In [ ]:
len(resultado)

In [ ]:
probabilidade_maior = np.argmax(resultado)
probabilidade_maior

In [ ]:
rede_neural_classificador.delete_endpoint()